In [90]:
import numpy as np
import random

# パラメータ設定
alpha = 0.0
beta = 1.0
customers = {0: 0.6, 5: 0.4}  # 顧客位置と需要
candidates = [1, 2, 3, 4]     # 候補地

def prob_leader(customer_pos, leader_pos, follower_pos, alpha=alpha, beta=beta):
    dL = abs(customer_pos - leader_pos)
    dF = abs(customer_pos - follower_pos)
    wL = np.exp(alpha - beta * dL)
    wF = np.exp(alpha - beta * dF)
    return wL / (wL + wF)

def leader_share(leader_pos, follower_pos):
    return sum(demand * prob_leader(cpos, leader_pos, follower_pos) for cpos, demand in customers.items())

def run_story(seed=None):
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)
    # 初期状態：リーダー／フォロワーともランダム（同一点は避ける）
    leader_init = random.choice(candidates)
    follower_init = random.choice([c for c in candidates if c != leader_init])
    init_share = leader_share(leader_init, follower_init)

    print(f"[初期状態] Leader がランダムに {leader_init} に配置")
    print(f"[初期状態] Follower がランダムに {follower_init} に配置 → Leaderシェア={init_share:.3f}")

    # フォロワを固定して、リーダが各点を選んだ場合のシェアを比較
    results = []
    print("\n[評価] Follower を固定したまま、Leader が各候補を選んだ場合のシェア：")
    for x in candidates:
        if x == follower_init:
            print(f"  Leader が {x} を選ぶ（不可：Followerと同一地点）→ スキップ")
            continue
        s = leader_share(x, follower_init)
        p0 = prob_leader(0, x, follower_init)
        p5 = prob_leader(5, x, follower_init)
        print(f"  Leader が {x} を選ぶ → シェア={s:.3f}（顧客0:{p0:.3f}, 顧客5:{p5:.3f}）")
        results.append((x, s))

    if not results:
        print("\n候補がありません（全てFollowerと重複）。")
        return

    # 同率の場合は小さい x を優先（好みで変更可）
    best_x, best_share = max(results, key=lambda t: (t[1], -t[0]))
    print("\n[結論] このため、Follower を固定した場合、Leader は "
          f"{best_x} を選ぶのが最適（シェア {best_share:.3f}）。")

# 実行（再現性が必要なら seed に整数を入れてください）
run_story(seed=None)


[初期状態] Leader がランダムに 2 に配置
[初期状態] Follower がランダムに 4 に配置 → Leaderシェア=0.576

[評価] Follower を固定したまま、Leader が各候補を選んだ場合のシェア：
  Leader が 1 を選ぶ → シェア=0.591（顧客0:0.953, 顧客5:0.047）
  Leader が 2 を選ぶ → シェア=0.576（顧客0:0.881, 顧客5:0.119）
  Leader が 3 を選ぶ → シェア=0.546（顧客0:0.731, 顧客5:0.269）
  Leader が 4 を選ぶ（不可：Followerと同一地点）→ スキップ

[結論] このため、Follower を固定した場合、Leader は 1 を選ぶのが最適（シェア 0.591）。


In [91]:
import numpy as np
import random

# パラメータ設定
alpha = 0.0
beta = 1.0
customers = {0: 0.6, 5: 0.4}
candidates = [1, 2, 3, 4]

def prob_leader(customer_pos, leader_pos, follower_pos, alpha=alpha, beta=beta):
    dL = abs(customer_pos - leader_pos)
    dF = abs(customer_pos - follower_pos)
    wL = np.exp(alpha - beta * dL)
    wF = np.exp(alpha - beta * dF)
    return wL / (wL + wF)

def leader_share(leader_pos, follower_pos):
    return sum(demand * prob_leader(cpos, leader_pos, follower_pos)
               for cpos, demand in customers.items())

def follower_share(leader_pos, follower_pos):
    # 2プレイヤ構造なので 1 - leader_share と等価だが、明示化
    return 1.0 - leader_share(leader_pos, follower_pos)

def run_full_story_follower_max(seed=None):
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)

    # --- 初期状態 ---
    leader_init = random.choice(candidates)
    follower_init = random.choice([c for c in candidates if c != leader_init])
    init_L = leader_share(leader_init, follower_init)
    init_F = 1 - init_L
    print(f"[初期状態] Leader がランダムに {leader_init} に配置")
    print(f"[初期状態] Follower がランダムに {follower_init} に配置 → Leaderシェア={init_L:.3f}, Followerシェア={init_F:.3f}")

    # --- Leaderの評価（Follower固定、Leaderが自分のシェアを最大化） ---
    results_leader = []
    print("\n[Leaderの評価] Follower を固定したまま、Leader が各候補を選んだ場合：")
    for x in candidates:
        if x == follower_init:
            print(f"  Leader が {x} を選ぶ（不可：Followerと同一地点）→ スキップ")
            continue
        Ls = leader_share(x, follower_init)
        Fs = 1 - Ls
        p0 = prob_leader(0, x, follower_init)
        p5 = prob_leader(5, x, follower_init)
        print(f"  Leader が {x} を選ぶ → Leader={Ls:.3f}, Follower={Fs:.3f}（顧客0:{p0:.3f}, 顧客5:{p5:.3f}）")
        results_leader.append((x, Ls))
    best_x, best_L = max(results_leader, key=lambda t: (t[1], -t[0]))
    best_F = 1 - best_L
    print(f"\n[Leader結論] このため、Leader は {best_x} を選ぶのが最適（Leader={best_L:.3f}, Follower={best_F:.3f}）")

    # --- Followerの評価（Leader固定、Followerが自分のシェアを最大化） ---
    results_follower = []
    print(f"\n[Followerの評価] Leader を {best_x} に固定した場合、Follower が各候補を選んだ場合：")
    for y in candidates:
        if y == best_x:
            print(f"  Follower が {y} を選ぶ（不可：Leaderと同一地点）→ スキップ")
            continue
        Ls = leader_share(best_x, y)
        Fs = 1 - Ls
        p0 = prob_leader(0, best_x, y)
        p5 = prob_leader(5, best_x, y)
        print(f"  Follower が {y} を選ぶ → Followerシェア={Fs:.3f}（Leader={Ls:.3f}｜顧客0:{p0:.3f}, 顧客5:{p5:.3f}）")
        results_follower.append((y, Fs))

    # Follower は自分のシェアを最大化（同率なら小さい y）
    best_y, best_F2 = max(results_follower, key=lambda t: (t[1], -t[0]))
    best_L2 = 1 - best_F2
    print(f"\n[Follower結論] このため、Follower は {best_y} を選ぶのが最適（Follower={best_F2:.3f}, Leader={best_L2:.3f}）")

    # --- 最終結果 ---
    print("\n=== 最終結果 ===")
    print(f"Leader: {best_x}, Follower: {best_y}, Leader={best_L2:.3f}, Follower={best_F2:.3f}")

# 実行（seed 指定で再現可）
run_full_story_follower_max(seed=None)


[初期状態] Leader がランダムに 1 に配置
[初期状態] Follower がランダムに 3 に配置 → Leaderシェア=0.576, Followerシェア=0.424

[Leaderの評価] Follower を固定したまま、Leader が各候補を選んだ場合：
  Leader が 1 を選ぶ → Leader=0.576, Follower=0.424（顧客0:0.881, 顧客5:0.119）
  Leader が 2 を選ぶ → Leader=0.546, Follower=0.454（顧客0:0.731, 顧客5:0.269）
  Leader が 3 を選ぶ（不可：Followerと同一地点）→ スキップ
  Leader が 4 を選ぶ → Leader=0.454, Follower=0.546（顧客0:0.269, 顧客5:0.731）

[Leader結論] このため、Leader は 1 を選ぶのが最適（Leader=0.576, Follower=0.424）

[Followerの評価] Leader を 1 に固定した場合、Follower が各候補を選んだ場合：
  Follower が 1 を選ぶ（不可：Leaderと同一地点）→ スキップ
  Follower が 2 を選ぶ → Followerシェア=0.454（Leader=0.546｜顧客0:0.731, 顧客5:0.269）
  Follower が 3 を選ぶ → Followerシェア=0.424（Leader=0.576｜顧客0:0.881, 顧客5:0.119）
  Follower が 4 を選ぶ → Followerシェア=0.409（Leader=0.591｜顧客0:0.953, 顧客5:0.047）

[Follower結論] このため、Follower は 2 を選ぶのが最適（Follower=0.454, Leader=0.546）

=== 最終結果 ===
Leader: 1, Follower: 2, Leader=0.546, Follower=0.454


In [92]:
import numpy as np
import random

alpha = 0.0
beta = 1.0
customers = {0: 0.6, 5: 0.4}
candidates = [1, 2, 3, 4]

def prob_leader(customer_pos, leader_pos, follower_pos, alpha=alpha, beta=beta):
    dL = abs(customer_pos - leader_pos)
    dF = abs(customer_pos - follower_pos)
    wL = np.exp(alpha - beta * dL)
    wF = np.exp(alpha - beta * dF)
    return wL / (wL + wF)

def leader_share(leader_pos, follower_pos):
    return sum(demand * prob_leader(cpos, leader_pos, follower_pos)
               for cpos, demand in customers.items())

def follower_share(leader_pos, follower_pos):
    return 1.0 - leader_share(leader_pos, follower_pos)

def best_leader_choice(follower_pos):
    results = []
    for x in candidates:
        if x == follower_pos:  # 同じ場所は不可
            continue
        Ls = leader_share(x, follower_pos)
        results.append((x, Ls))
    return max(results, key=lambda t: (t[1], -t[0]))  # 最大シェア

def best_follower_choice(leader_pos):
    results = []
    for y in candidates:
        if y == leader_pos:
            continue
        Fs = follower_share(leader_pos, y)
        results.append((y, Fs))
    return max(results, key=lambda t: (t[1], -t[0]))  # 最大シェア

def run_repeated_decisions(max_steps=10, seed=None):
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)

    # 初期状態ランダム
    leader = random.choice(candidates)
    follower = random.choice([c for c in candidates if c != leader])
    print(f"[初期] Leader={leader}, Follower={follower}, "
          f"L={leader_share(leader,follower):.3f}, F={follower_share(leader,follower):.3f}")

    history = [(leader, follower)]

    for step in range(1, max_steps+1):
        if step % 2 == 1:  # Leader のターン
            leader, Ls = best_leader_choice(follower)
            Fs = 1 - Ls
            print(f"[{step}手目: Leader] Leader={leader}, Follower={follower}, L={Ls:.3f}, F={Fs:.3f}")
        else:  # Follower のターン
            follower, Fs = best_follower_choice(leader)
            Ls = 1 - Fs
            print(f"[{step}手目: Follower] Leader={leader}, Follower={follower}, L={Ls:.3f}, F={Fs:.3f}")

        # 収束チェック
        if (leader, follower) in history:
            print(f"\n[収束検出] ({leader}, {follower}) に戻ったので停止。")
            break
        history.append((leader, follower))

    print("\n=== 最終結果 ===")
    print(f"Leader={leader}, Follower={follower}, "
          f"L={leader_share(leader,follower):.3f}, F={follower_share(leader,follower):.3f}")

# 実行（例）
run_repeated_decisions(max_steps=10, seed=None)


[初期] Leader=3, Follower=4, L=0.546, F=0.454
[1手目: Leader] Leader=1, Follower=4, L=0.591, F=0.409
[2手目: Follower] Leader=1, Follower=2, L=0.546, F=0.454
[3手目: Leader] Leader=1, Follower=2, L=0.546, F=0.454

[収束検出] (1, 2) に戻ったので停止。

=== 最終結果 ===
Leader=1, Follower=2, L=0.546, F=0.454


In [93]:
import numpy as np
import random

# パラメータ
alpha = 0.0
beta = 1.0
customers = {0: 0.6, 5: 0.4}
candidates = [1, 2, 3, 4]

def prob_leader(customer_pos, leader_pos, follower_pos, alpha=alpha, beta=beta):
    dL = abs(customer_pos - leader_pos)
    dF = abs(customer_pos - follower_pos)
    wL = np.exp(alpha - beta * dL)
    wF = np.exp(alpha - beta * dF)
    return wL / (wL + wF)

def leader_share(leader_pos, follower_pos):
    return sum(demand * prob_leader(cpos, leader_pos, follower_pos)
               for cpos, demand in customers.items())

def follower_share(leader_pos, follower_pos):
    return 1.0 - leader_share(leader_pos, follower_pos)

def leader_eval_block(fixed_follower):
    # Leaderが各候補を試す（Follower固定）
    results = []
    print(f"[Leaderの評価] Follower を {fixed_follower} に固定したまま、Leader が各候補を選んだ場合：")
    for x in candidates:
        if x == fixed_follower:
            print(f"  Leader が {x} を選ぶ（不可：Followerと同一地点）→ スキップ")
            continue
        Ls = leader_share(x, fixed_follower)
        Fs = 1 - Ls
        p0 = prob_leader(0, x, fixed_follower)
        p5 = prob_leader(5, x, fixed_follower)
        print(f"  Leader が {x} を選ぶ → Leader={Ls:.3f}, Follower={Fs:.3f}（顧客0:{p0:.3f}, 顧客5:{p5:.3f}）")
        results.append((x, Ls))
    best_x, best_L = max(results, key=lambda t: (t[1], -t[0]))
    best_F = 1 - best_L
    print(f"[Leader結論] このため、Leader は {best_x} を選ぶのが最適（Leader={best_L:.3f}, Follower={best_F:.3f}）\n")
    return best_x, best_L

def follower_eval_block(fixed_leader):
    # Followerが各候補を試す（Leader固定）
    results = []
    print(f"[Followerの評価] Leader を {fixed_leader} に固定した場合、Follower が各候補を選んだ場合：")
    for y in candidates:
        if y == fixed_leader:
            print(f"  Follower が {y} を選ぶ（不可：Leaderと同一地点）→ スキップ")
            continue
        Ls = leader_share(fixed_leader, y)
        Fs = 1 - Ls
        p0 = prob_leader(0, fixed_leader, y)
        p5 = prob_leader(5, fixed_leader, y)
        print(f"  Follower が {y} を選ぶ → Followerシェア={Fs:.3f}（Leader={Ls:.3f}｜顧客0:{p0:.3f}, 顧客5:{p5:.3f}）")
        results.append((y, Fs))
    best_y, best_F = max(results, key=lambda t: (t[1], -t[0]))
    best_L = 1 - best_F
    print(f"[Follower結論] このため、Follower は {best_y} を選ぶのが最適（Follower={best_F:.3f}, Leader={best_L:.3f}）\n")
    return best_y, best_F

def run_until_convergence(max_steps=10, seed=None):
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)

    # 初期：ランダム（同一点は避ける）
    leader_pos = random.choice(candidates)
    follower_pos = random.choice([c for c in candidates if c != leader_pos])
    print(f"[初期状態] Leader がランダムに {leader_pos} に配置")
    print(f"[初期状態] Follower がランダムに {follower_pos} に配置 → "
          f"Leader={leader_share(leader_pos,follower_pos):.3f}, Follower={follower_share(leader_pos,follower_pos):.3f}\n")

    history = [(leader_pos, follower_pos)]

    for step in range(1, max_steps+1):
        print(f"=== {step} 手目 ===")
        # Leaderの最適反応
        leader_pos, _ = leader_eval_block(fixed_follower=follower_pos)
        # Followerの最適反応
        follower_pos, _ = follower_eval_block(fixed_leader=leader_pos)

        pair = (leader_pos, follower_pos)
        if pair in history:
            print(f"[収束検出] 既出の組 ({leader_pos}, {follower_pos}) に到達。停止します。")
            break
        history.append(pair)

    print("\n=== 最終結果 ===")
    L_final = leader_share(leader_pos, follower_pos)
    F_final = 1 - L_final
    print(f"Leader={leader_pos}, Follower={follower_pos}, Leader={L_final:.3f}, Follower={F_final:.3f}")

# 実行（収束まで観察）
run_until_convergence(max_steps=10, seed=None)


[初期状態] Leader がランダムに 4 に配置
[初期状態] Follower がランダムに 3 に配置 → Leader=0.454, Follower=0.546

=== 1 手目 ===
[Leaderの評価] Follower を 3 に固定したまま、Leader が各候補を選んだ場合：
  Leader が 1 を選ぶ → Leader=0.576, Follower=0.424（顧客0:0.881, 顧客5:0.119）
  Leader が 2 を選ぶ → Leader=0.546, Follower=0.454（顧客0:0.731, 顧客5:0.269）
  Leader が 3 を選ぶ（不可：Followerと同一地点）→ スキップ
  Leader が 4 を選ぶ → Leader=0.454, Follower=0.546（顧客0:0.269, 顧客5:0.731）
[Leader結論] このため、Leader は 1 を選ぶのが最適（Leader=0.576, Follower=0.424）

[Followerの評価] Leader を 1 に固定した場合、Follower が各候補を選んだ場合：
  Follower が 1 を選ぶ（不可：Leaderと同一地点）→ スキップ
  Follower が 2 を選ぶ → Followerシェア=0.454（Leader=0.546｜顧客0:0.731, 顧客5:0.269）
  Follower が 3 を選ぶ → Followerシェア=0.424（Leader=0.576｜顧客0:0.881, 顧客5:0.119）
  Follower が 4 を選ぶ → Followerシェア=0.409（Leader=0.591｜顧客0:0.953, 顧客5:0.047）
[Follower結論] このため、Follower は 2 を選ぶのが最適（Follower=0.454, Leader=0.546）

=== 2 手目 ===
[Leaderの評価] Follower を 2 に固定したまま、Leader が各候補を選んだ場合：
  Leader が 1 を選ぶ → Leader=0.546, Follower=0.454（顧客0:0.731, 顧客5:0.269）
  Leader

In [94]:
import numpy as np

# パラメータ
alpha = 0.0   # 魅力度
beta = 1.0    # 距離に対する感度
customers = {1: 0.6, 6: 0.4}   # 顧客の位置と需要
candidates = [2, 3, 4, 5]      # 候補地

def prob_leader(customer_pos, leader_pos, follower_pos, alpha=alpha, beta=beta):
    """顧客がLeaderを選ぶ確率"""
    dL = abs(customer_pos - leader_pos)
    dF = abs(customer_pos - follower_pos)
    wL = np.exp(alpha - beta * dL)
    wF = np.exp(alpha - beta * dF)
    return wL / (wL + wF)

def leader_share(leader_pos, follower_pos):
    """Leaderの総シェア"""
    return sum(demand * prob_leader(cpos, leader_pos, follower_pos) for cpos, demand in customers.items())

def follower_share(leader_pos, follower_pos):
    """Followerの総シェア（Leaderの補数）"""
    return 1.0 - leader_share(leader_pos, follower_pos)

# Leaderが候補地を選んだとき、それぞれのFollowerの反応をシミュレーション
def simulate(leader_choice):
    print(f"\n=== Leader が {leader_choice} を選んだ場合 ===")
    for f in candidates:
        if f == leader_choice:
            continue
        Ls = leader_share(leader_choice, f)
        Fs = 1 - Ls
        print(f"Follower が {f} を選ぶ → Leaderシェア={Ls:.3f}, Followerシェア={Fs:.3f}")

# 実行例: Leader が候補地ごとに選んだ場合
for leader_choice in candidates:
    simulate(leader_choice)



=== Leader が 2 を選んだ場合 ===
Follower が 3 を選ぶ → Leaderシェア=0.546, Followerシェア=0.454
Follower が 4 を選ぶ → Leaderシェア=0.576, Followerシェア=0.424
Follower が 5 を選ぶ → Leaderシェア=0.591, Followerシェア=0.409

=== Leader が 3 を選んだ場合 ===
Follower が 2 を選ぶ → Leaderシェア=0.454, Followerシェア=0.546
Follower が 4 を選ぶ → Leaderシェア=0.546, Followerシェア=0.454
Follower が 5 を選ぶ → Leaderシェア=0.576, Followerシェア=0.424

=== Leader が 4 を選んだ場合 ===
Follower が 2 を選ぶ → Leaderシェア=0.424, Followerシェア=0.576
Follower が 3 を選ぶ → Leaderシェア=0.454, Followerシェア=0.546
Follower が 5 を選ぶ → Leaderシェア=0.546, Followerシェア=0.454

=== Leader が 5 を選んだ場合 ===
Follower が 2 を選ぶ → Leaderシェア=0.409, Followerシェア=0.591
Follower が 3 を選ぶ → Leaderシェア=0.424, Followerシェア=0.576
Follower が 4 を選ぶ → Leaderシェア=0.454, Followerシェア=0.546
